In [ ]:
#Importação das bibliotecas
import gcsfs 
import intake
import xarray as xr
import pandas as pd
import cftime
import geopandas as gpd
import regionmask
import re
import numpy as np
import fiona
import xesmf as xe
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
#Níveis do WOA para comparação: 
new_levels = [0.00e+00, 5.00e+00, 1.00e+01, 1.50e+01, 2.00e+01, 2.50e+01,
       3.00e+01, 3.50e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01,
       6.00e+01, 6.50e+01, 7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01,
       9.00e+01, 9.50e+01, 1.00e+02, 1.25e+02, 1.50e+02, 1.75e+02,
       2.00e+02, 2.25e+02, 2.50e+02, 2.75e+02, 3.00e+02, 3.25e+02,
       3.50e+02, 3.75e+02, 4.00e+02, 4.25e+02, 4.50e+02, 4.75e+02,
       5.00e+02, 5.50e+02, 6.00e+02, 6.50e+02, 7.00e+02, 7.50e+02,
       8.00e+02, 8.50e+02, 9.00e+02, 9.50e+02, 1.00e+03, 1.05e+03,
       1.10e+03, 1.15e+03, 1.20e+03, 1.25e+03, 1.30e+03, 1.35e+03,
       1.40e+03, 1.45e+03, 1.50e+03, 1.55e+03, 1.60e+03, 1.65e+03,
       1.70e+03, 1.75e+03, 1.80e+03, 1.85e+03, 1.90e+03, 1.95e+03,
       2.00e+03, 2.10e+03, 2.20e+03, 2.30e+03, 2.40e+03, 2.50e+03,
       2.60e+03, 2.70e+03, 2.80e+03, 2.90e+03, 3.00e+03, 3.10e+03,
       3.20e+03, 3.30e+03, 3.40e+03, 3.50e+03, 3.60e+03, 3.70e+03,
       3.80e+03, 3.90e+03, 4.00e+03, 4.10e+03, 4.20e+03, 4.30e+03,
       4.40e+03, 4.50e+03, 4.60e+03, 4.70e+03, 4.80e+03, 4.90e+03,
       5.00e+03, 5.10e+03, 5.20e+03, 5.30e+03, 5.40e+03, 5.50e+03]


In [ ]:
#Função para acertar a variável tempo! Pois alguns formatos disponiveis no CMIP6 para a variável time dificultam sua manipulação. 
def to_360day_monthly(da):
    ''' Conversão da dimensão de tempo de modelos climáticos.
        Função criada por Claire Carouge no CLEX CMS Blog'''
    val = da.copy()
    time1 = da.time.copy()
    for itime in range(val.sizes['time']):
        bb = val.time.values[itime].timetuple()
        time1.values[itime] = cftime.Datetime360Day(bb[0],bb[1],16)
    val = val.assign_coords({'time':time1})
    return val

#Função que converte a profundidade de centímetros para metro.
def depth_m(ds):
    if "lev" in ds:
        if "units" in ds["lev"].attrs:
            units = ds["lev"].units.lower()
            if units == "cm" or units == "centimeters":
                ds["lev"] = ds["lev"] / 100
                ds["lev"].attrs["units"] = "m"
    return ds

In [ ]:
#Acessa o conjunto de metadados do CMIP6 hospedados no Google Cloud.
cmip6 = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [ ]:
#Lendo a minha Tabela de Modelo.
#Verifique o caminho, no meu jupyter lab está tudo na mesma pasta.
df = pd.read_excel("Tabela_Modelos/Tabela_Modelos.xlsx", sheet_name=1)
df = df.rename(columns = lambda x : x.strip())

In [ ]:
gdf = gpd.read_file('Area_Projeto/Regiao_Juntas/Regiao_Juntas.shp')

In [ ]:
#Realiza minha pesquisa de acordo com a minha Tabela de Modelos
pesquisas = []
for index, row in df.iterrows():
    pesquisa = {'source_id': row['source_id'],
                'table_id': row['table_id'],
                'variable_id': row['variable_id'].split(', '),
                'experiment_id': row['experiment_id'],
                'member_id': row['member_id']}
    pesquisas.append(pesquisa)

In [ ]:
import pandas as pd

for pesquisa in pesquisas:
    cat = cmip6.search(require_all_on='source_id', **pesquisa)
    cat = cat.to_dataset_dict(aggregate=True,
                              storage_options={'token': 'anon'},
                              xarray_open_kwargs={'consolidated': True,
                                                 'decode_times': True,
                                                 'use_cftime': True})
    
    # Variável para armazenar o dataset
    ds = None

    # Iterar sobre as chaves da lista
    for key in list(cat.keys()):
        # Verificar se a chave termina com "gr"
        if key.endswith("gr"):
            ds = cat[key]
            break  # Se encontrou "gr", não é necessário continuar procurando

    # Se "gr" não foi encontrado, ds ficará com o último valor atribuído no loop
    # Se nenhum valor termina com "gr", ds ficará com o último valor terminado com "gn"
    if ds is None:
        for key in list(cat.keys()):
            if key.endswith("gn"):
                ds = cat[key]
                break  # Se encontrou "gn", não é necessário continuar procurando

    #Converte o time para o formato correto.
    ds = ds[["thetao", "so"]].squeeze()
    
    if "gn" in ds.grid_label:
        ds_out = xr.Dataset(
            {
                "lat" : (["lat"], np.arange(-90, 90, 1.0)),
                "lon" : (["lon"], np.arange(-180, 180, 1.0)),
            }
        )

        regridder = xe.Regridder(ds, ds_out, "bilinear", ignore_degenerate=True)
        ds_out = regridder(ds, keep_attrs = True)
    
    else:
        ds_out = ds
    
    #Renomeia olevel para level
    ds_out = rename_coords(ds_out)
    
    # Interpolação vertical usando interpolação linear
    ds_interp = ds_out.interp(lev=new_levels, method='linear')
    
    #Converte o GeoDataFrame para um objeto region mask
    mask = regionmask.mask_geopandas(gdf, ds_interp['lon'], ds_interp['lat'])

    #Aplica a máscara ao dataset
    ds_masked = ds_interp.where(mask==mask, drop=True)
    
    ds_lat = ds_masked.mean(dim="lon")
    
    ds_lat.to_netcdf("Hov_Lat/{}.nc".format(key))